In [ ]:
import pandas as pd
from vresutils.costdata import annuity
import yaml

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    config = yaml.safe_load(stream)["costs"]

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    elec_config = yaml.safe_load(stream)["electricity"]

In [ ]:
with open("../data/nicenames.yaml", 'r') as stream:
    nn = yaml.safe_load(stream)

In [ ]:
Nyears = 1
opt_fn = "../data/costs_optimist.csv"
pes_fn = "../data/costs_pessimist.csv"
peur_fn = "../data/costs_pypsaeur.csv"

In [ ]:
def load_costs(fn):

    # set all asset costs and other parameters
    costs = pd.read_csv(fn, index_col=[0,1]).sort_index()

    # correct units to kW
    costs.loc[costs.unit.str.contains("/MW"),"value"] /= 1e3
    costs.unit = costs.unit.str.replace("/MW", "/kW")

    fill_values = config["fill_values"]
    costs = costs.value.unstack().fillna(fill_values)

    costs["capital_cost"] = ((annuity(costs["lifetime"], costs["discount rate"]) +
                             costs["FOM"]/100.) *
                             costs["investment"] * Nyears)

    costs['marginal_cost'] = costs['VOM']

    costs.at['solar', 'capital_cost'] = config["rooftop_share"] * costs.at['solar-rooftop', 'capital_cost'] + \
                                        (1-config["rooftop_share"]) * costs.at['solar-utility', 'capital_cost']

    def costs_for_storage(store, link1, link2=None, max_hours=1.):        
        
        capital_cost = link1['capital_cost'] + max_hours * store['capital_cost']
        if link2 is not None:
            capital_cost += link2['capital_cost']
        return pd.Series(dict(capital_cost=capital_cost,
                              marginal_cost=0.,
                              co2_emissions=0.))

    max_hours = elec_config['max_hours']
    costs.loc["battery"] = \
        costs_for_storage(costs.loc["battery storage"], costs.loc["battery inverter"],
                          max_hours=max_hours['battery'])
    costs.loc["H2"] = \
        costs_for_storage(costs.loc["hydrogen storage tank"], costs.loc["fuel cell"],
                          costs.loc["electrolysis"], max_hours=max_hours['H2']) 
    return costs

In [ ]:
opt = load_costs(opt_fn)
pes = load_costs(pes_fn)
peur = load_costs(peur_fn)

In [ ]:
cost_uncertainty = pd.concat([opt.capital_cost, pes.capital_cost, peur.capital_cost],
                              keys=["optimist", "pessimist", "pypsaeur"], axis=1)
keep = ["battery", "H2", "solar", "offwind", "onwind"]
cost_uncertainty = cost_uncertainty.loc[keep].round()

In [ ]:
cost_uncertainty.to_csv("cost-uncertainty.csv")

In [ ]:
regex = '(HVAC|HVDC|PHS|battery |electrolysis|fuel cell|hydro$|hydrogen storage tank|offwind|onwind|ror|solar-)'
drop = ["capital_cost", "marginal_cost", "fuel", "discount rate", "VOM"]
optf = opt.filter(regex=regex, axis=0).drop(drop, axis=1)
pesf = pes.filter(regex=regex, axis=0).drop(drop, axis=1)
peurf = peur.filter(regex=regex, axis=0).drop(drop+["CO2 intensity"], axis=1)

In [ ]:
df = optf.copy()

df["investment pes"] = pesf.investment.round(1)
df["investment peur"] = peurf.investment.round(1)
df.investment = df.investment.round(1)

df.lifetime = df.lifetime.astype(int)
df.loc[df.efficiency==1.,"efficiency"] = "-"
df.loc[df.FOM==0.,"FOM"] = "-"

df.drop(["HVDC inverter pair", "HVDC submarine"], axis=0, inplace=True)
df = df[["FOM", "efficiency", "lifetime", "investment", "investment pes", "investment peur"]]

df.rename(index=nn["rows"], columns=nn["columns"], inplace=True)
df.sort_index(ascending=False, inplace=True)

In [ ]:
df.to_csv("cost-table.csv")

In [ ]:
with open('cost-table.tex', 'w') as tf:
     tf.write(df.to_latex())